In [1]:
import sys
import warnings
sys.path.append('../')
warnings.filterwarnings('ignore')

%load_ext autoreload
%autoreload 1
%aimport utils, GroupingAlgorithm, HEEM_VQE_Functions

In [129]:
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
from tqdm.auto import tqdm

from qiskit import IBMQ, Aer, QuantumCircuit, QuantumRegister, ClassicalRegister, transpile
from qiskit.providers.aer import QasmSimulator
from qiskit.providers.aer.noise import NoiseModel
from qiskit.utils.quantum_instance import QuantumInstance
from qiskit.ignis.verification import get_ghz_simple
from qiskit.compiler import transpile
from qiskit.tools.monitor import job_monitor
from qiskit.opflow.state_fns import CircuitStateFn
from qiskit.providers.aer.extensions.snapshot_statevector import *

from utils import get_backend_connectivity, molecules, Label2Chain, from_string_to_numbers, load_grouping_data
from GroupingAlgorithm import groupingWithOrder, TPBgrouping
from HEEM_VQE_Functions import measure_circuit_factor, probability2expected_binary, post_process_results

In [3]:
def compute_energy(Groups, Measurements, method, n_chunks=10, layout=None):
    
    circuits = [measure_circuit_factor(measurement, n_qubits, measure_all=False).compose(state_0, front=True) for measurement
                 in Measurements]
    
    qi = QuantumInstance(backend=backed_calculations, coupling_map=coupling_map, noise_model=noise_model,
                          basis_gates=basis_gates, shots=NUM_SHOTS // len(Groups))
    
    if layout is not None:
        qi.set_config(initial_layout=layout[::-1])
        
    qi._RUN_CONFIG += ["job_name", "job_tags"]
    qi.set_config(job_name=f'{molecule_name} / {method}')

    counts = []

    counter = 0
    n_runs = int(np.ceil(len(circuits) / n_chunks))
    
    with open('progress_' + method + '.txt', 'w') as f:
            f.write('{}/{}\n'.format(0, len(circuits)))
    
    pbar = tqdm(total=len(circuits), desc='Simulating circuits for {}'.format(method))
    while len(counts) < len(circuits):
        initial = counter * n_chunks
        final = min((counter + 1) * n_chunks, len(circuits))
         
        # Some time IBMQ is returning some random error [5204], so if this happens
        while True:
            try:
                qi.set_config(job_tags=[f'{counter + 1}/{n_runs}'])
                counts_temp = qi.execute(circuits[initial:final]).get_counts()
                break
            except (KeyboardInterrupt, SystemExit):
                return None
            except Exception as e:
                print(e)
                print('Trying again ...')
                pass

        counts += counts_temp
        counter += 1
        
        with open('progress_' + method + '.txt', 'w') as f:
            f.write('{}/{}\n'.format(final, len(circuits)))            
        pbar.update(final - initial)
    pbar.close()
    
    if len(circuits) == 1:
        counts = [counts]
        
    probabilities = [post_process_results(counts[j], circuits[j].num_clbits, qi.run_config.shots) for j in
                     range(len(counts))]
    energy = 0
    
    for j in range(len(probabilities)):
        diagonals, factors = probability2expected_binary(coeffs, labels, [Groups[j]], [Measurements[j]], shift=False)
        diagonals = [~diagonal * 2 - 1 for diagonal in diagonals[0]]

        energy += np.sum((diagonals * np.array(factors[0])[:, None]) @ probabilities[j])
        # energy += np.sum(prob2Exp[j] @ probabilities[j])

    return qi, energy

In [4]:
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    IBMQ.load_account()

provider_main = IBMQ.get_provider(hub='ibm-q', group='open', project='main')
provider_CSIC = IBMQ.get_provider(hub='ibm-q-csic', group='internal', project='iff-csic')

name_backend = 'ibmq_montreal'
backend = provider_CSIC.get_backend(name_backend)
WC_device = get_backend_connectivity(backend)

G_device = nx.Graph()
G_device.add_edges_from(WC_device)

In [5]:
NUM_SHOTS = 2 ** 14

backend_hpc = provider_main.get_backend('ibmq_qasm_simulator')
backed_simulation = provider_main.get_backend('simulator_statevector')
simulator = Aer.get_backend('aer_simulator')  # Backend for simulation

device = QasmSimulator.from_backend(backend)
coupling_map = device.configuration().coupling_map
noise_model = NoiseModel.from_backend(device)
basis_gates = noise_model.basis_gates

backed_calculations = simulator

In [168]:
molecule_name = 'CH3OH'

try:
    qubit_op = np.load('../data/big_molecules.npy', allow_pickle=True).item()[molecule_name]
    print('Data loaded')
except KeyError:
    print('Computing molecule')
    qubit_op = molecules(molecule_name)
        
paulis, coeffs, labels = Label2Chain(qubit_op)

n_qubits = qubit_op.num_qubits
state_0 = QuantumCircuit(n_qubits)
# state_0 = state_0.compose(get_ghz_simple(n_qubits, measure=False))  # Initialized in the GHZ state

print(f'{len(paulis)} total Pauli strings')

Data loaded
9257 total Pauli strings


# Exact

# Exact (Simulation)

In [169]:
Groups, Measurements, _ = load_grouping_data(molecule_name, 'EM')
print('Number of groups: {}'.format(len(Groups)))

circuits_exact = []
for j, measure in enumerate(Measurements):
    circuit = measure_circuit_factor(measure, n_qubits, make_measurements=False)
    circuit = circuit.compose(state_0, front=True)
    # circuit.save_statevector()
    circuit.snapshot_statevector('snapshot')
    circuits_exact.append(circuit)

Data loaded
Number of groups: 682


In [ ]:
n_max = 300

if n_max == -1:
    n_max = len(circuits_exact)
n_runs = int(np.ceil(len(circuits_exact) / n_max))

results = []

pbar = tqdm(total=len(circuits_exact), desc='Simulating circuits')
for i in range(n_runs):
    initial = i * n_max
    final = min(initial + n_max, len(circuits_exact))
    
    results += backed_simulation.run(circuits_exact[initial:final], shots=1).result().to_dict()['results']
    pbar.update(final - initial)
pbar.close()

Simulating circuits:   0%|          | 0/682 [00:00<?, ?it/s]

In [ ]:
pbar = tqdm(total=len(Measurements), desc='Computing energy')
energy_exact = 0
for j in range(len(Measurements)):
    diagonals, factors = probability2expected_binary(coeffs, labels, [Groups[j]], [Measurements[j]], shift=True)
    diagonals = [~diagonal * 2 - 1 for diagonal in diagonals[0]]
    
    prob = np.linalg.norm(np.array(results[j]['data']['snapshots']['statevector']['snapshot'][0]), axis=1) ** 2    
    energy_exact += np.sum((diagonals * np.array(factors[0])[:, None]) @ prob)
    pbar.update()
pbar.close()
    
print('Exact energy: {}'.format(energy_exact))

# TPB

In [ ]:
_, Groups_TPB, Measurements_TPB = TPBgrouping(paulis, print_progress=True)

print('Number of groups', len(Groups_TPB))

In [ ]:
method = 'TPB'
qi_TPB, energy_TPB = compute_energy(Groups_TPB, Measurements_TPB, method, n_chunks=100)
relative_TPB = np.abs((energy_TPB - energy_exact) / energy_exact)

print('TPB energy: {}'.format(energy_TPB))
print('Relative error: {:.3f} %'.format(relative_TPB * 100))
print(f'Simulation time {qi_TPB.time_taken} s')

with open('progress_' + method + '.txt', 'w') as f:
    f.write('Done: {}\n'.format(energy_TPB))

# EM

In [ ]:
Groups_EM, Measurements_EM, layout_EM = groupingWithOrder(paulis, connected=True, print_progress=True)

print('Number of groups', len(Groups_EM))

In [ ]:
method = 'EM'

qi_EM, energy_EM = compute_energy(Groups_EM, Measurements_EM, method, layout=layout_EM, n_chunks=25)
relative_EM= np.abs((energy_EM - energy_exact) / energy_exact)

print('EM energy: {}'.format(energy_EM))
print('Relative error: {:.3f} %'.format(relative_EM * 100))
print(f'Simulation time {qi_EM.time_taken} s')

with open('progress_' + method + '.txt', 'w') as f:
    f.write('Done: {}\n'.format(energy_EM))

# HEEM

In [ ]:
Groups_HEEM, Measurements_HEEM, layout_HEEM = groupingWithOrder(paulis, G_device, connected=True, print_progress=True)

print('Number of groups', len(Groups_HEEM))

In [ ]:
method = 'HEEM'

qi_HEEM, energy_HEEM = compute_energy(Groups_HEEM, Measurements_HEEM, method, n_chunks=100, layout=layout_HEEM)
relative_HEEM = np.abs((energy_HEEM - energy_exact) / energy_exact)

print('HEEM energy: {}'.format(energy_HEEM))
print('Relative error: {:.3f} %'.format(relative_HEEM * 100))
print(f'Simulation time {qi_HEEM.time_taken} s')

with open('progress_' + method + '.txt', 'w') as f:
    f.write('Done: {}\n'.format(energy_HEEM))